In [31]:
import pandas as pd
import altair as alt
url = 'https://raw.githubusercontent.com/xh0417/IV_group63/main/europe_covid.csv'
data = pd.read_csv(url)

In [32]:
# Select the top 10 countries with the highest number of active cases
attribute = ['Top-10 Active Cases', 'Top-10 Total Cases', 'Top-10 Total Deaths']

top10_active = pd.DataFrame({
    'Country': data.nlargest(10, 'Active Cases')['Country'],
    'Value': data.nlargest(10, 'Active Cases')['Active Cases'],
    'Attribute': 'Top-10 Active Cases'
})

top10_TotalCases = pd.DataFrame({
    'Country': data.nlargest(10, 'Total Cases')['Country'],
    'Value': data.nlargest(10, 'Total Cases')['Total Cases'],
    'Attribute': 'Top-10 Total Cases'
})

top10_TotalDeaths = pd.DataFrame({
    'Country': data.nlargest(10, 'Total Deaths')['Country'],
    'Value': data.nlargest(10, 'Total Deaths')['Total Deaths'],
    'Attribute': 'Top-10 Total Deaths'
})
all_stats = pd.concat([top10_active, top10_TotalCases, top10_TotalDeaths], axis=0)

In [33]:
def normalize(x):
    return x / max(x)
data = pd.read_csv('https://raw.githubusercontent.com/xh0417/IV_group63/main/europe_covid.csv')
data["Total Cases"] = (data["Total Cases"] - data["Total Cases"].min()) / (data["Total Cases"].max() - data["Total Cases"].min())
data["Total Deaths"] = (data["Total Deaths"] - data["Total Deaths"].min()) / (data["Total Deaths"].max() - data["Total Deaths"].min())
data["Total Recovered"] = (data["Total Recovered"] - data["Total Recovered"].min()) / (data["Total Recovered"].max() - data["Total Recovered"].min())
data["Active Cases"] = (data["Active Cases"] - data["Active Cases"].min()) / (data["Active Cases"].max() - data["Active Cases"].min())
data["Tot Cases/ 1M pop"] = (data["Tot Cases/ 1M pop"] - data["Tot Cases/ 1M pop"].min()) / (data["Tot Cases/ 1M pop"].max() - data["Tot Cases/ 1M pop"].min())
data["Deaths/ 1M pop"] = (data["Deaths/ 1M pop"] - data["Deaths/ 1M pop"].min()) / (data["Deaths/ 1M pop"].max() - data["Deaths/ 1M pop"].min())
data["Total Tests"] = (data["Total Tests"] - data["Total Tests"].min()) / (data["Total Tests"].max() - data["Total Tests"].min())
data["Tests/ 1M pop"] = (data["Tests/ 1M pop"] - data["Tests/ 1M pop"].min()) / (data["Tests/ 1M pop"].max() - data["Tests/ 1M pop"].min())
data["Population"] = (data["Population"] - data["Population"].min()) / (data["Population"].max() - data["Population"].min())

df = pd.melt(data, id_vars=["Country"], value_vars=["Total Cases","Total Deaths","Total Recovered","Active Cases","Tot Cases/ 1M pop","Deaths/ 1M pop","Total Tests","Tests/ 1M pop","Population"], var_name="x_axis", value_name="value")


In [34]:
#brush = alt.selection_interval(
#    encodings=['y'] # limit selection to x-axis (year) values
#)
brush = alt.selection_single(
    encodings=['y'],
    on='mouseover'
)

In [35]:
selection = alt.selection_single(
    name='Select',
    fields=['Attribute'],
    init={'Attribute': 'Top-10 Active Cases'},
    bind={'Attribute': alt.binding_radio(options=attribute)}
)

In [36]:
bars = alt.Chart().mark_bar().encode(
    x='Value:Q',
    y=alt.Y('Country:N', sort='-x'),
    color=alt.Color("Country:N")
).properties(
    width=400, height=300,
    title='COVID-19 Statistics by Country'
)


In [37]:
# Filter the data based on the select box value
bars_filtered = bars.transform_filter(
    selection
).properties(
    title='COVID-19 Statistics by Country'
)


In [38]:
# Combine the chart and data
chart = alt.layer(
    bars_filtered,
    data=all_stats
).add_selection(
    selection,brush
)

chart2 = alt.Chart(df).mark_line().encode(
    x=alt.X("x_axis:N", sort=alt.EncodingSortField(field="index", order="ascending")),
    y=alt.Y("value:Q"),
    color=alt.Color("Country:N"),
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(width=400, height=300)

In [39]:
br = alt.hconcat(chart, chart2).properties(spacing=5)
br.save('bursh_linking2.html')